# Assignment 2 - Question 4: Convolutional Neural Network
**Course Name:** Machine Learning (DDA3020)

<font color=Red>*Please enter your personal information (Double-click this block first)*</font>

**Name:** CAI, Yuquan

**Student ID:** 122090007

**It's highly recommended to finish Question 3 first.**

### Overview

In this question, you will implement two CNN models and train them on the same dataset as Question 3 (Fasion-MNIST). We will discover how well-suited CNNs are for intensive data tasks such as image processing, compared to traditional machine learning algorithms (like those tree tree-based models in Question 3). Similarly, your task is to **run all codes in this script and complete the parts marked with** <font color=Red>\[TASK\]</font>.

### Introduction of TensorFlow

TensorFlow is a powerful open-source package for machine learning and deep learning, enabling efficient implementation of complex models like neural networks with ease. First of all, you need to install the TensorFlow package with the version of 2.9

```bash
pip install numpy==1.26 tensorflow==2.9 -i https://mirrors.aliyun.com/pypi/simple/ 
```

by running this commend in your command line window. To check whether the package is successfully installed, you can try to run the following import block. If you're struggling with installing TensorFlow, you can upload the coding assignment to Google Colab for execution or seek assistance from TA.

In [1]:
import numpy as np
import gzip
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

Don't need to carefully read this block since it's just loading the dataset. Just run it.

In [2]:
def load_mnist(path, kind, subset=None):
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte.gz'%kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte.gz'%kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)
    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)
    
    if subset is not None:
        selected_images, selected_labels = [], []
        for label in range(10):
            indices = np.where(labels == label)[0]
            selected_indices = np.random.choice(indices, subset, replace=False)
            selected_images.append(images[selected_indices])
            selected_labels.append(labels[selected_indices])
        images = np.concatenate(selected_images, axis=0)
        labels = np.concatenate(selected_labels, axis=0)

        paired = list(zip(images, labels))
        random.shuffle(paired)
        images, labels = zip(*paired)
    
    return np.array(images), np.array(labels)

In this question, we will use all data of Fashion-MNIST and do a little bit data preprocessing.

In [3]:
X_train, y_train = load_mnist('./data/', kind='train')
X_test, y_test = load_mnist('./data/', kind='t10k')

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

If you are familiar with TensorFlow or PyTorch, you can straightly start Task 1. Otherwise, it's highly recommended to start from https://www.tensorflow.org/tutorials/images/cnn to gain some basic capacity on building neural network by TensorFlow.

### Task 1

At the beginning, we need to build a very simple CNN model with the structure of
1. A 2D convolutional layer with 16 filters with each size 3*3 (RELU activation function)
2. A 2D maxpooling layer with 2*2 pooling window
3. A flatten layer to convert 2D feature into 1D vector
4. A fully connected layer using Softmax activation

Remember that we are doing a image classification task, so we shall use categorical cross entropy function as the loss funtion. <font color=Red>\[TASK\]</font> (10 points)

In [4]:
##########################
## Write your code here ##
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
##########################


model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.1)  # Train the model
test_loss, test_acc = model.evaluate(X_test, y_test)                        #  Test the model

Epoch 1/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4784 - accuracy: 0.8351 - val_loss: 0.3718 - val_accuracy: 0.8677
Epoch 2/2
313/313 [==============================] - 0s 984us/step - loss: 0.3252 - accuracy: 0.8861


### Task 2

Then we will take a challenge to implement a more complex CNN model to have a better classification performance. Here is a structure for your reference, or you can also design your own CNN model. The only requirement is to have a better performance than the simple CNN in Task 1 (a larger accuracy score on test set).

The reference structure is devided into three parts:
1. Primary Feature Extraction Part
    1. A 2D convolutional layer with 32 filters with each size 3*3 (RELU activation function)
    2. A normalization layer
    3. A 2D maxpooling layer with 2*2 pooling window
    4. A dropout layer (randomly drops 25% of units) (designed for preventing overfitting)
2. Advanced Feature Extraction Part

    This part is mostly similar to the previous section. The only difference is to use more filters (like 64) in convolutional layer to gain high-level features
3. Classification Part
    1. A flatten layer to convert 2D feature into 1D vector
    2. A fully connected layer with 512 units using RELU to summerize high-dimensinal features
    3. Another connected layer for Softmax classification

Remember that we are doing a image classification task, so we shall use categorical cross entropy function as the loss funtion. <font color=Red>\[TASK\]</font> (10 points)

In [6]:
##########################
## Write your code here ##
model = models.Sequential([
    # 1. Primary Feature Extraction Part
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # 2. Advanced Feature Extraction Part
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # 3. Classification Part
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
##########################


model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.1)
test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/2
1688/1688 [==============================] - 13s 7ms/step - loss: 0.4493 - accuracy: 0.8410 - val_loss: 0.3219 - val_accuracy: 0.8843
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 0.3180 - accuracy: 0.8793
